In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from utils import to_grayscale_then_rgb

In [2]:
file_train_b_csv='..\Iris_train_basic_seg_list.csv'
df_train_basic= pd.read_csv(file_train_b_csv)
datagen=ImageDataGenerator(rescale=1./255.,preprocessing_function=to_grayscale_then_rgb)
df_train_basic['Label']= df_train_basic['Label'].astype(str)
df_train_basic


,Name,Label
0,110160_OD_INF_SANO.jpg,0
1,110160_OD_NAS_SANO.jpg,0
2,110160_OD_SUP_SANO.jpg,0
3,110160_OD_TEM_SANO.jpg,0
4,2581150_OD_INF_CMV.jpg,1
...,...,...
123,781208_OI_TEM_SANO.jpg,0
124,795004_OI_INF_POSTNER.jpg,1
125,795004_OI_NAS_POSTNER.jpg,1
126,795004_OI_SUP_POSTNER.jpg,1


In [3]:
file_val_csv='..\Iris_val_basic_seg_list.csv'
df_val= pd.read_csv(file_val_csv)
df_val['Label']= df_val['Label'].astype(str)


In [4]:

train_generator = datagen.flow_from_dataframe(
    dataframe=df_train_basic,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\train_basic",
    x_col="Name",
    y_col="Label",
    batch_size=5,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224))#540,810


Found 128 validated image filenames belonging to 2 classes.


In [5]:
val_generator = datagen.flow_from_dataframe(
    dataframe=df_val,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\val",
    x_col="Name",
    y_col="Label",
    batch_size=5,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224))


Found 32 validated image filenames belonging to 2 classes.


In [ ]:
file_test_csv='..\Iris_test_seg_list.csv'
df_test= pd.read_csv(file_test_csv)
df_test['class_label']= df_test['class_label'].astype(str)
df_test['label_OI']= df_test['label_OI'].astype(str)
df_test['label_OD']= df_test['label_OD'].astype(str)
test_generator_OI = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\test",
    x_col="OI",
    y_col="label_OI",
    batch_size=2,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(224, 224))

In [6]:
from model import build_LeNet5
input_shape=(224,224,1)
model=build_LeNet5(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

In [7]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
#STEP_SIZE_TEST=test_generator_OI.n//test_generator_OI.batch_size
#model.reset_states()

In [8]:
hist=model.fit(x=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=val_generator, 
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,verbose=1)

Epoch 1/20
25/25 [==============================] - 11s 297ms/step - loss: 0.7148 - accuracy: 0.4960 - val_loss: 0.6946 - val_accuracy: 0.3667
Epoch 2/20
25/25 [==============================] - 6s 255ms/step - loss: 0.6346 - accuracy: 0.7805 - val_loss: 0.7636 - val_accuracy: 0.5000
Epoch 3/20
25/25 [==============================] - 7s 292ms/step - loss: 0.4336 - accuracy: 0.8211 - val_loss: 0.8197 - val_accuracy: 0.5000
Epoch 4/20
25/25 [==============================] - 12s 460ms/step - loss: 0.1507 - accuracy: 0.9593 - val_loss: 2.1571 - val_accuracy: 0.4333
Epoch 5/20
25/25 [==============================] - 4s 178ms/step - loss: 0.0718 - accuracy: 0.9837 - val_loss: 1.4298 - val_accuracy: 0.4333
Epoch 6/20
25/25 [==============================] - 4s 153ms/step - loss: 0.0215 - accuracy: 0.9919 - val_loss: 1.9714 - val_accuracy: 0.4667
Epoch 7/20
25/25 [==============================] - 4s 166ms/step - loss: 0.0726 - accuracy: 0.9756 - val_loss: 1.8837 - val_accuracy: 0.4667
Epoc

In [ ]:
model.evaluate(x=test_generator_OI,
steps=STEP_SIZE_TEST)
